##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TF2 워크플로에서 TF1.x 모델 사용하기


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/model_mapping"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.org에서 보기</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/model_mapping.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/model_mapping.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 보기</a> </td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/model_mapping.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

이 가이드는 즉시 실행, `tf.function`, 모델링 코드 변경을 최소화하는 배포 전략과 같이 TF2 워크플로에서 기존 TF1.x 모델을 사용하는 데 사용할 수 있는 [모델링 코드 shim](https://en.wikipedia.org/wiki/Shim_(computing))의 개요와 예제를 제공합니다.

## 사용 범위

이 가이드에 설명된 shim은 다음에 의존하는 TF1.x 모델에서 사용할 수 있게 설계되어 있습니다.

1. 변수 생성 및 재사용을 제어하는 `tf.compat.v1.get_variable` 및 `tf.compat.v1.variable_scope`
2. 가중치 및 정규화 손실을 추적하는 `tf.compat.v1.global_variables()`, `tf.compat.v1.trainable_variables`, `tf.compat.v1.losses.get_regularization_losses()` 및 `tf.compat.v1.get_collection()`

여기에는 `tf.compat.v1.layer`, `tf.contrib.layers` API, [TensorFlow-Slim](https://github.com/google-research/tf-slim)을 기반으로 구축된 대부분의 모델이 포함됩니다.

다음 TF1.x 모델에는 shim이 필요하지 **않습니다**.

1. `model.trainable_weights` 및 `model.losses`를 통해 이미 훈련할 수 있는 모든 가중치와 정규화 손실을 추적하는 독립형 Keras 모델
2. `module.trainable_variables`를 통해 훈련할 수 있는 모든 가중치를 이미 추적하고 아직 생성되지 않은 경우에만 가중치를 생성하는 `tf.Module`

이러한 모델들은 즉시 실행 및 `tf.function`을 사용하여 TF2에서 작동할 가능성이 높습니다.

## 설치하기

TensorFlow 및 기타 종속성을 가져옵니다.

In [ ]:
!pip uninstall -y -q tensorflow

In [ ]:
# Install tf-nightly as the DeterministicRandomTestTool is available only in
# Tensorflow 2.8

!pip install -q tf-nightly

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as v1
import sys
import numpy as np

from contextlib import contextmanager

## `track_tf1_style_variables` 데코레이터

이 가이드에서 설명하는 주요 shim은 TF1.x-style 가중치를 추적하고 정규화 손실을 캡처하는 `tf.keras.layers.Layer`와 `tf.Module`에 속한 메서드 내에서 사용할 수 있는 데코레이터인 `tf.compat.v1.keras.utils.track_tf1_style_variables`입니다.

`tf.compat.v1.keras.utils.track_tf1_style_variables`를 사용하여 `tf.keras.layers.Layer` 또는 `tf.Module`의 호출 메서드를 데코레이팅하면 호출될 때마다 항상 새 변수를 생성하는 대신 데코레이팅된 메서드 내에서 올바르게 작동하는 `tf.compat.v1.get_variable`(및 확장자 `tf.compat.v1.layers`)를 통해 변수를 생성하고 재사용할 수 있습니다. 또한 레이어 또는 모듈이 데코레이팅된 메서드 내에서 `get_variable`을 통해 생성되거나 액세스된 가중치를 암시적으로 추적하도록 합니다.

표준 `layer.variable`/`module.variable`/etc 속성에서 가중치 자체를 추적하는 것 외에도 메서드가 `tf.keras.layers.Layer`에 속하면 `get_variable` 또는 `tf.compat.v1.layers` 정규화 인수를 통해 지정된 정규화 손실은 표준 `layer.losses` 속성 아래의 레이어에서 추적됩니다.

이 추적 메커니즘을 사용하면 TF2 작동이 사용 설정된 경우에도 Keras 레이어 또는 TF2의 `tf.Module` 내부에서 TF1.x-style 모델 순방향 전달 코드의 큰 클래스를 사용할 수 있습니다.


## 사용 예제

아래의 사용 예제는 `tf.keras.layers.Layer` 메서드를 데코레이팅할 때 사용하는 모델링 shim을 보여주지만, Keras 특성과 구체적으로 상호작용하는 경우를 제외하고 `tf.Module`을 데코레이팅할 때에도 적용할 수 있습니다.

### tf.compat.v1.get_variable를 사용하여 빌드한 레이어

다음과 같이 `tf.compat.v1.get_variable`를 기반으로 직접 구현한 레이어가 있다고 상상해 보겠습니다.

```python
def dense(self, inputs, units):
  out = inputs
  with tf.compat.v1.variable_scope("dense"):
    # The weights are created with a `regularizer`,
    kernel = tf.compat.v1.get_variable(
        shape=[out.shape[-1], units],
        regularizer=tf.keras.regularizers.L2(),
        initializer=tf.compat.v1.initializers.glorot_normal,
        name="kernel")
    bias = tf.compat.v1.get_variable(
        shape=[units,],
        initializer=tf.compat.v1.initializers.zeros,
        name="bias")
    out = tf.linalg.matmul(out, kernel)
    out = tf.compat.v1.nn.bias_add(out, bias)
  return out
```

Shim을 레이어로 변환한 후 입력에서 호출합니다.

In [ ]:
class DenseLayer(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    out = inputs
    with tf.compat.v1.variable_scope("dense"):
      # The weights are created with a `regularizer`,
      # so the layer should track their regularization losses
      kernel = tf.compat.v1.get_variable(
          shape=[out.shape[-1], self.units],
          regularizer=tf.keras.regularizers.L2(),
          initializer=tf.compat.v1.initializers.glorot_normal,
          name="kernel")
      bias = tf.compat.v1.get_variable(
          shape=[self.units,],
          initializer=tf.compat.v1.initializers.zeros,
          name="bias")
      out = tf.linalg.matmul(out, kernel)
      out = tf.compat.v1.nn.bias_add(out, bias)
    return out

layer = DenseLayer(10)
x = tf.random.normal(shape=(8, 20))
layer(x)

표준 Keras 레이어와 같은 추적된 변수와 캡처된 정규화 손실에 액세스합니다.

In [ ]:
layer.trainable_variables
layer.losses

레이어를 호출할 때마다 가중치가 재사용되는지 확인하려면 모든 가중치를 0으로 설정하고 레이어를 다시 호출합니다.

In [ ]:
print("Resetting variables to zero:", [var.name for var in layer.trainable_variables])

for var in layer.trainable_variables:
  var.assign(var * 0.0)

# Note: layer.losses is not a live view and
# will get reset only at each layer call
print("layer.losses:", layer.losses)
print("calling layer again.")
out = layer(x)
print("layer.losses: ", layer.losses)
out

Keras 함수형 모델 구성에서도 변환한 레이어를 직접 사용할 수 있습니다.

In [ ]:
inputs = tf.keras.Input(shape=(20))
outputs = DenseLayer(10)(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

x = tf.random.normal(shape=(8, 20))
model(x)

# Access the model variables and regularization losses
model.weights
model.losses

### `tf.compat.v1.layers`로 빌드한 모델

다음과 같이 `tf.compat.v1.layers`를 기반으로 직접 구현한 레이어가 있다고 상상해 보겠습니다.

```python
def model(self, inputs, units):
  with tf.compat.v1.variable_scope('model'):
    out = tf.compat.v1.layers.conv2d(
        inputs, 3, 3,
        kernel_regularizer="l2")
    out = tf.compat.v1.layers.flatten(out)
    out = tf.compat.v1.layers.dense(
        out, units,
        kernel_regularizer="l2")
    return out
```

Shim을 레이어로 변환한 후 입력에서 호출합니다.

In [ ]:
class CompatV1LayerModel(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    with tf.compat.v1.variable_scope('model'):
      out = tf.compat.v1.layers.conv2d(
          inputs, 3, 3,
          kernel_regularizer="l2")
      out = tf.compat.v1.layers.flatten(out)
      out = tf.compat.v1.layers.dense(
          out, self.units,
          kernel_regularizer="l2")
      return out

layer = CompatV1LayerModel(10)
x = tf.random.normal(shape=(8, 5, 5, 5))
layer(x)

경고: 안전상의 이유로 모든 `tf.compat.v1.layers`를 비어 있지 않은 문자열 `variable_scope` 안에 넣어야 합니다. 자동 생성된 이름이 있는 `tf.compat.v1.layers`는 항상 변수 범위 밖에서 이름을 자동으로 늘리기 때문입니다. 즉, 레이어/모듈을 호출할 때마다 요청된 변수 이름이 일치하지 않습니다. 따라서 이미 만든 가중치를 재사용하는 대신 호출할 때마다 새로운 변수 세트를 생성합니다.

표준 Keras 레이어와 같은 추적된 변수와 캡처된 정규화 손실에 액세스합니다.

In [ ]:
layer.trainable_variables
layer.losses

레이어를 호출할 때마다 가중치가 재사용되는지 확인하려면 모든 가중치를 0으로 설정하고 레이어를 다시 호출합니다.

In [ ]:
print("Resetting variables to zero:", [var.name for var in layer.trainable_variables])

for var in layer.trainable_variables:
  var.assign(var * 0.0)

out = layer(x)
print("layer.losses: ", layer.losses)
out

Keras 함수 모델 구성에서도 변환한 레이어를 직접 사용할 수 있습니다.

In [ ]:
inputs = tf.keras.Input(shape=(5, 5, 5))
outputs = CompatV1LayerModel(10)(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

x = tf.random.normal(shape=(8, 5, 5, 5))
model(x)

In [ ]:
# Access the model variables and regularization losses
model.weights
model.losses

### 배치 정규화 업데이트 및 모델 `training` 인수 캡처하기

TF1.x에서는 다음과 같이 배치 정규화를 수행합니다.

```python
  x_norm = tf.compat.v1.layers.batch_normalization(x, training=training)

  # ...

  update_ops = tf.compat.v1.get_collection(tf.GraphKeys.UPDATE_OPS)
  train_op = optimizer.minimize(loss)
  train_op = tf.group([train_op, update_ops])
```

참고:

1. 배치 정규화 이동 평균 업데이트는 레이어와는 별개로 호출된 `get_collection`에 의해 추적됩니다.
2. `tf.compat.v1.layers.batch_normalization`에는 `training` 인수가 필요합니다(TF-Slim 배치 정규화 레이어를 사용하는 경우 일반적으로 `is_training`이라고 함).

TF2에서는 [즉시 실행](https://www.tensorflow.org/guide/eager)과 자동 제어 종속성으로 인해 배치 정규화 이동 평균 업데이트가 즉시 실행됩니다. 업데이트 컬렉션에서 별도로 수집하고 명시적 제어 종속성으로 추가할 필요가 없습니다.

또한 `tf.keras.layers.Layer`의 순방향 전달 메서드에 `training` 인수를 지정하면 Keras는 다른 레이어에서 하는 것처럼 현재 훈련 단계와 모든 중첩 레이어를 전달할 수 있게 됩니다. Keras가 `training` 인수를 처리하는 방법에 대한 자세한 정보는 `tf.keras.Model`용 API 문서를 참조하세요.

`tf.Module` 메서드를 데코레이팅하는 경우에는 필요에 따라 모든 `training` 인수를 수동으로 전달해야 합니다. 그러나 배치 정규화 이동 평균 업데이트는 명시적인 제어 종속성이 없어도 여전히 자동으로 적용됩니다.

다음 코드 조각은 shim에 배치 정규화 레이어를 삽입하는 방법과 Keras 모델에서 이를 사용하는 방법을 보여줍니다(`tf.keras.layers.Layer`에 적용 가능).

In [ ]:
class CompatV1BatchNorm(tf.keras.layers.Layer):

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    print("Forward pass called with `training` =", training)
    with v1.variable_scope('batch_norm_layer'):
      return v1.layers.batch_normalization(x, training=training)

In [ ]:
print("Constructing model")
inputs = tf.keras.Input(shape=(5, 5, 5))
outputs = CompatV1BatchNorm()(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

print("Calling model in inference mode")
x = tf.random.normal(shape=(8, 5, 5, 5))
model(x, training=False)

print("Moving average variables before training: ",
      {var.name: var.read_value() for var in model.non_trainable_variables})

# Notice that when running TF2 and eager execution, the batchnorm layer directly
# updates the moving averages while training without needing any extra control
# dependencies
print("calling model in training mode")
model(x, training=True)

print("Moving average variables after training: ",
      {var.name: var.read_value() for var in model.non_trainable_variables})


### 변수 범위 기반 변수 재사용하기

`get_variable`을 기반으로 하는 순방향 전달에서 생성한 모든 변수는 TF1.x의 변수 범위와 동일한 변수 이름 지정과 재사용 의미 체계를 유지합니다. 위에서 언급한 것처럼 자동 생성한 이름을 가진 모든 `tf.compat.v1.layers`에 대해 비어 있지 않은 외부 범위가 하나 이상 갖는 한 참입니다.

참고: 이름 지정 및 재사용은 단일 레이어/모듈 인스턴스 내로 범위가 지정됩니다. 하나의 shim 데코레이션 레이어 또는 모듈 내에서 `get_variable`을 호출하면 레이어나 모듈 내에서 생성한 변수를 참조할 수 없습니다. 필요한 경우 `get_variable`을 통해 변수에 액세스하는 대신 Python 참조를 다른 변수에 직접 사용하여 이 문제를 해결할 수 있습니다.

### 즉시 실행과 `tf.function`

위에서 보았듯이 `tf.keras.layers.Layer` 및 `tf.Module`로 데코레이팅된 메서드는 즉시 실행 내부에서 실행되며 `tf.function`과도 호환됩니다. 즉, [pdb](https://docs.python.org/3/library/pdb.html) 및 기타 대화형 도구를 사용하여 실행 중인 순방향 전달을 단계별로 실행할 수 있음을 의미합니다.

경고: `tf.function`의 *내부*에서 shim으로 데코레이트한 레이어/모듈 메서드를 호출하는 것은 완벽하게 안전하지만 `tf.functions`에 `get_variable` 호출이 포함된 경우 `tf.function`를 넣는 것은 안전하지 않습니다. `tf.function`을 입력하면 `variable_scope`가 재설정됩니다. 즉, shim이 모방하는 TF1.x 스타일 변수 범위 기반의 변수 재사용이 이 설정에서 중단됩니다.

### 분산 전략

`@track_tf1_style_variables`로 데코레이트한 레이어 또는 모듈 메서드 안에서 이루어지는 `get_variable` 호출은 내부에서 표준 `tf.Variable` 변수 생성을 사용합니다. 즉, `MirroredStrategy` 및 `TPUStrategy`와 같은 `tf.distribute`에서 사용할 수 있는 다양한 분산 전략과 함께 사용할 수 있습니다.

## 데코레이션 호출에서 `tf.Variable`, `tf.Module`, `tf.keras.layers` 및 `tf.keras.models` 중첩하기

`tf.compat.v1.keras.utils.track_tf1_style_variables`에서 레이어 호출을 데코레이팅하면 `tf.compat.v1.get_variable`을 통해 생성(및 재사용)한 변수의 자동 암시 추적만 추가됩니다. 일반적인 Keras 레이어 및 대부분의 `tf.Module`에서 사용하는 것과 같이 `tf.Variable` 호출로 직접 생성한 가중치는 캡처하지 않습니다. 이 섹션에서는 이러한 중첩 사례를 처리하는 방법을 설명합니다.


### (기존 사용법) `tf.keras.layers` 및 `tf.keras.models`

중첩 Keras 레이어 및 모델의 기존 사용법에서는 `tf.compat.v1.keras.utils.get_or_create_layer`를 사용합니다. 이는 기존 TF1.x 중첩 Keras 사용을 쉽게 마이그레이션할 때만 권장됩니다. 새 코드는 tf.Variables 및 tf.Modules에 대해 아래에 설명한 대로 명시적 속성 설정을 사용해야 합니다.

`tf.compat.v1.keras.utils.get_or_create_layer`를 사용하려면 중첩 모델을 구성하는 코드를 메서드로 래핑한 후 메서드에 전달해야 합니다. 예제:

In [ ]:
class NestedModel(tf.keras.Model):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units

  def build_model(self):
    inp = tf.keras.Input(shape=(5, 5))
    dense_layer = tf.keras.layers.Dense(
        10, name="dense", kernel_regularizer="l2",
        kernel_initializer=tf.compat.v1.ones_initializer())
    model = tf.keras.Model(inputs=inp, outputs=dense_layer(inp))
    return model

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    # Get or create a nested model without assigning it as an explicit property
    model = tf.compat.v1.keras.utils.get_or_create_layer(
        "dense_model", self.build_model)
    return model(inputs)

layer = NestedModel(10)
layer(tf.ones(shape=(5,5)))

이 메서드는 이러한 중첩 레이어가 TensorFlow에서 올바르게 재사용되고 추적되도록 합니다. `@track_tf1_style_variables` 데코레이터는 적절한 메서드에서 여전히 필요합니다. `get_or_create_layer`(이 경우 `self.build_model`)로 전달된 모델 빌더 메서드는 인수를 사용하면 안 됩니다.

가중치는 다음과 같이 추적합니다.

In [ ]:
assert len(layer.weights) == 2
weights = {x.name: x for x in layer.variables}

assert set(weights.keys()) == {"dense/bias:0", "dense/kernel:0"}

layer.weights

정규화 손실도 다음과 같습니다.

In [ ]:
tf.add_n(layer.losses)

### 증분 마이그레이션: `tf.Variables` 및 `tf.Modules`

데코레이팅한 메서드에 `tf.Variable` 호출 또는 `tf.Module`을 삽입해야 하는 경우(예: 이 가이드의 뒷부분에 설명된 레거시가 아닌 TF2 API로의 증분 마이그레이션을 따르는 경우) 다음 요구 사항에 따라 명시적으로 추적해야 합니다.

- 변수/모듈/레이어가 한 번만 생성되었는지 명시적으로 확인
- [일반 모듈 또는 레이어](https://www.tensorflow.org/guide/intro_to_modules#defining_models_and_layers_in_tensorflow)를 정의할 때와 마찬가지로 인스턴스 속성으로 명시적으로 연결
- 후속 호출에서 이미 생성한 객체를 명시적으로 재사용

이렇게 하면 호출할 때마다 가중치를 새롭게 생성하지 않고 올바르게 재사용합니다. 또한 이를 통해 기존 가중치 및 정규화 손실을 추적할 수 있습니다.

다음은 이러한 작동 방식을 설명한 예제입니다.

In [ ]:
class NestedLayer(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def __call__(self, inputs):
    out = inputs
    with tf.compat.v1.variable_scope("inner_dense"):
      # The weights are created with a `regularizer`,
      # so the layer should track their regularization losses
      kernel = tf.compat.v1.get_variable(
          shape=[out.shape[-1], self.units],
          regularizer=tf.keras.regularizers.L2(),
          initializer=tf.compat.v1.initializers.glorot_normal,
          name="kernel")
      bias = tf.compat.v1.get_variable(
          shape=[self.units,],
          initializer=tf.compat.v1.initializers.zeros,
          name="bias")
      out = tf.linalg.matmul(out, kernel)
      out = tf.compat.v1.nn.bias_add(out, bias)
    return out

class WrappedDenseLayer(tf.keras.layers.Layer):

  def __init__(self, units, **kwargs):
    super().__init__(**kwargs)
    self.units = units
    # Only create the nested tf.variable/module/layer/model
    # once, and then reuse it each time!
    self._dense_layer = NestedLayer(self.units)

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    with tf.compat.v1.variable_scope('outer'):
      outputs = tf.compat.v1.layers.dense(inputs, 3)
      outputs = tf.compat.v1.layers.dense(inputs, 4)
      return self._dense_layer(outputs)

layer = WrappedDenseLayer(10)

layer(tf.ones(shape=(5, 5)))

`track_tf1_style_variables` 데코레이터로 데코레이팅한 경우에도 중첩 모듈을 명시적으로 추적해야 합니다. 이는 데코레이팅한 메서드가 있는 각 모듈/레이어에 연결된 자체 변수 저장소가 있기 때문입니다.

다음과 같은 경우 가중치를 올바르게 추적합니다.

In [ ]:
assert len(layer.weights) == 6
weights = {x.name: x for x in layer.variables}

assert set(weights.keys()) == {"outer/inner_dense/bias:0",
                               "outer/inner_dense/kernel:0",
                               "outer/dense/bias:0",
                               "outer/dense/kernel:0",
                               "outer/dense_1/bias:0",
                               "outer/dense_1/kernel:0"}

layer.trainable_weights

정규화 손실도 마찬가지입니다.

In [ ]:
layer.losses

`NestedLayer`가 Keras가 아닌 `tf.Module`인 경우 변수는 계속 추적되지만 정규화 손실은 자동으로 추적되지 않으므로 명시적으로 따로따로 추적해야 할 수 있습니다.

### 변수 이름 안내

명시적 `tf.Variable` 호출 및 Keras 레이어는 `get_variable` 및 `variable_scopes`의 조합에서 사용하는 것과는 다른 레이어 이름/변수 이름 자동 생성 메커니즘을 사용합니다. shim은 TF1.x 그래프에서 TF2 즉시 실행 및 `tf.function`으로 이동하는 경우에도 `get_variable`으로 생성한 변수 이름이 일치되게 만들지만, `tf.Variable` 호출과 메서드 데코레이터 내 임베딩한 Keras 레이어를 대상으로 생성한 변수 이름과의 일치 여부는 보장할 수 없습니다. 여러 변수가 TF2 즉시 실행 및 `tf.function`에서 동일한 이름을 공유할 수도 있습니다.

이 가이드의 뒷부분에서 정확성 검증 및 TF1.x 체크포인트 매핑에 대한 섹션을 진행할 때 특히 주의해야 합니다.

### 데코레이팅된 메서드에서 `tf.compat.v1.make_template` 사용하기

**`tf.compat.v1.make_template`을 사용하는 대신 TF2에서 레이어가 더 얇은 `tf.compat.v1.keras.utils.track_tf1_style_variables`를 직접 사용하는 것을 권장합니다.**.

이미 `tf.compat.v1.make_template`에 의존하고 있던 이전 TF1.x 코드의 내용은 이 섹션의 안내를 따르세요.

`tf.compat.v1.make_template`은 `get_variable`을 사용하는 코드를 래핑하므로 `track_tf1_style_variables` 데코레이터를 사용하면 레이어 호출에서 이러한 템플릿을 사용하고 가중치 및 정규화 손실을 추적할 수 있습니다.

단, `make_template`을 한 번만 호출한 다음 각 레이어 호출에서 동일한 템플릿을 재사용해야 합니다. 그렇지 않으면 새 변수 세트로 레이어를 호출할 때마다 새 템플릿이 생성됩니다.

예제는 다음과 같습니다.

In [ ]:
class CompatV1TemplateScaleByY(tf.keras.layers.Layer):

  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    def my_op(x, scalar_name):
      var1 = tf.compat.v1.get_variable(scalar_name,
                            shape=[],
                            regularizer=tf.compat.v1.keras.regularizers.L2(),
                            initializer=tf.compat.v1.constant_initializer(1.5))
      return x * var1
    self.scale_by_y = tf.compat.v1.make_template('scale_by_y', my_op, scalar_name='y')

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    with tf.compat.v1.variable_scope('layer'):
      # Using a scope ensures the `scale_by_y` name will not be incremented
      # for each instantiation of the layer.
      return self.scale_by_y(inputs)

layer = CompatV1TemplateScaleByY()

out = layer(tf.ones(shape=(2, 3)))
print("weights:", layer.weights)
print("regularization loss:", layer.losses)
print("output:", out)

경고: shim 데코레이터의 변수 및 정규화 손실 추적 메커니즘이 손상될 수 있으므로 여러 레이어 인스턴스에서 동일한 `make_template` 생성 템플릿을 공유하지 않는 것이 좋습니다. 또한 여러 레이어 인스턴스 내에서 동일한 `make_template` 이름을 사용하려면 `variable_scope` 내에서 생성한 템플릿을 중첩해서 사용해야 합니다. 그렇지 않으면 템플릿의 `variable_scope`에 대해 생성한 이름이 레이어의 새 인스턴스마다 증가합니다. 이로 인해 예상치 못한 방식으로 가중치 이름이 변경될 수 있습니다.

## 네이티브 TF2로 증분 마이그레이션

앞서 언급했듯이 `track_tf1_style_variables`를 사용하면 TF2 스타일 객체 지향 `tf.Variable`/`tf.keras.layers.Layer`/`tf.Module` 사용을 동일한 데코레이션 모듈/레이어 내부에서 레거시 `tf.compat.v1.get_variable`/`tf.compat.v1.layers` 스타일 사용과 혼합할 수 있습니다.

즉, TF1.x 모델이 TF2와 완전히 호환되도록 만든 후 네이티브(`tf.compat.v1`이 아닌) TF2 API를 사용하여 모든 새 모델 구성 요소를 작성하고 이전 코드를 상호 운용할 수 있습니다.

다만, 이전 모델 구성 요소를 계속 수정하는 경우 레거시 스타일 `tf.compat.v1` 사용을 새로 작성한 TF2 코드에 권장되는 순수 네이티브 객체 지향 API로 점진적으로 전환하도록 선택할 수도 있습니다.

Keras 레이어/모델을 데코레이팅하는 경우에는 `self.add_weight` 호출로, Keras 객체 또는 `tf.Module`을 데코레이팅하는 경우 `tf.Variable` 호출로 `tf.compat.v1.get_variable` 사용을 교체할 수 있습니다.

함수형 스타일 및 객체 지향 `tf.compat.v1.layers`는 일반적으로 인수를 변경하지 않아도 동등한 `tf.keras.layers` 레이어로 교체할 수 있습니다.

또한 `track_tf1_style_variables`를 사용할 수 있는 순수 네이티브 API로 점진적으로 이동하는 동안 모델이나 일반 패턴의 청크 부분을 개별 레이어/모듈로 변경하는 방안을 고려할 수도 있습니다.

### Slim 및 contrib.layers에 대한 노트

다수의 기존 TF 1.x 코드는 TF 1.x와 함께 `tf.contrib.layers`로 패키징된 [Slim](https://ai.googleblog.com/2016/08/tf-slim-high-level-library-to-define.html) 라이브러리를 사용합니다. Slim을 사용하여 코드를 네이티브 TF 2로 변환하는 작업은 `v1.layers`를 변환하는 작업보다 더 복잡합니다. 사실 Slim 코드를 먼저 `v1.layers`로 변환한 다음 Keras로 변환하는 것이 합리적일 수 있습니다. 다음은 Slim 코드를 변환하는 몇 가지 일반적인 가이드입니다.

- 모든 인수가 명시적인지 확인합니다. 가능한 경우 `arg_scopes`를 제거합니다. 계속 사용해야 하는 경우 `normalizer_fn` 및 `activation_fn`을 자체 레이어로 분할합니다.
- 분리할 수 있는 전환 레이어는 하나 이상의 다른 Keras 레이어(깊이별, 포인트별 및 분리 가능한 Keras 레이어)에 매핑됩니다.
- Slim과 `v1.layers`는 인수 이름과 기본값이 다릅니다.
- 일부 인수는 다른 행렬을 사용합니다.

### 체크포인트 호환성을 무시하고 네이티브 TF2로 마이그레이션하기

다음 코드 샘플은 체크포인트 호환성을 고려하지 않고 모델을 순수 네이티브 API로 점진적으로 이동하는 방법을 보여줍니다.

In [ ]:
class CompatModel(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    with tf.compat.v1.variable_scope('model'):
      out = tf.compat.v1.layers.conv2d(
          inputs, 3, 3,
          kernel_regularizer="l2")
      out = tf.compat.v1.layers.flatten(out)
      out = tf.compat.v1.layers.dropout(out, training=training)
      out = tf.compat.v1.layers.dense(
          out, self.units,
          kernel_regularizer="l2")
      return out


그런 다음 `compat.v1` API를 해당하는 네이티브 객체 지향 API로 부분적으로 교체합니다. 컨볼루션 레이어를 레이어 생성자에서 생성한 Keras 객체로 전환하여 시작합니다.

In [ ]:
class PartiallyMigratedModel(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units
    self.conv_layer = tf.keras.layers.Conv2D(
      3, 3,
      kernel_regularizer="l2")

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    with tf.compat.v1.variable_scope('model'):
      out = self.conv_layer(inputs)
      out = tf.compat.v1.layers.flatten(out)
      out = tf.compat.v1.layers.dropout(out, training=training)
      out = tf.compat.v1.layers.dense(
          out, self.units,
          kernel_regularizer="l2")
      return out


[`v1.keras.utils.DeterministicRandomTestTool`](https://www.tensorflow.org/api_docs/python/tf/compat/v1/keras/utils/DeterministicRandomTestTool) 클래스를 사용하여 이 증분을 변경해도 모델이 이전과 동일하게 작동하는지 확인합니다.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  tf.keras.utils.set_random_seed(42)
  layer = CompatModel(10)

  inputs = tf.random.normal(shape=(10, 5, 5, 5))
  original_output = layer(inputs)

  # Grab the regularization loss as well
  original_regularization_loss = tf.math.add_n(layer.losses)

print(original_regularization_loss)

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  tf.keras.utils.set_random_seed(42)
  layer = PartiallyMigratedModel(10)

  inputs = tf.random.normal(shape=(10, 5, 5, 5))
  migrated_output = layer(inputs)

  # Grab the regularization loss as well
  migrated_regularization_loss = tf.math.add_n(layer.losses)

print(migrated_regularization_loss)

In [ ]:
# Verify that the regularization loss and output both match
np.testing.assert_allclose(original_regularization_loss.numpy(), migrated_regularization_loss.numpy())
np.testing.assert_allclose(original_output.numpy(), migrated_output.numpy())

이제 모든 개별 `compat.v1.layers`를 네이티브 Keras 레이어로 교체했습니다.

In [ ]:
class NearlyFullyNativeModel(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units
    self.conv_layer = tf.keras.layers.Conv2D(
      3, 3,
      kernel_regularizer="l2")
    self.flatten_layer = tf.keras.layers.Flatten()
    self.dense_layer = tf.keras.layers.Dense(
      self.units,
      kernel_regularizer="l2")

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    with tf.compat.v1.variable_scope('model'):
      out = self.conv_layer(inputs)
      out = self.flatten_layer(out)
      out = self.dense_layer(out)
      return out


In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  tf.keras.utils.set_random_seed(42)
  layer = NearlyFullyNativeModel(10)

  inputs = tf.random.normal(shape=(10, 5, 5, 5))
  migrated_output = layer(inputs)

  # Grab the regularization loss as well
  migrated_regularization_loss = tf.math.add_n(layer.losses)

print(migrated_regularization_loss)

In [ ]:
# Verify that the regularization loss and output both match
np.testing.assert_allclose(original_regularization_loss.numpy(), migrated_regularization_loss.numpy())
np.testing.assert_allclose(original_output.numpy(), migrated_output.numpy())

마지막으로, 남아 있는(더 이상 필요하지 않은) `variable_scope` 사용과 `track_tf1_style_variables` 데코레이터를 모두 제거합니다.

이제 완전히 네이티브 API를 사용하는 모델 버전만 남았습니다.

In [ ]:
class FullyNativeModel(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units
    self.conv_layer = tf.keras.layers.Conv2D(
      3, 3,
      kernel_regularizer="l2")
    self.flatten_layer = tf.keras.layers.Flatten()
    self.dense_layer = tf.keras.layers.Dense(
      self.units,
      kernel_regularizer="l2")

  def call(self, inputs):
    out = self.conv_layer(inputs)
    out = self.flatten_layer(out)
    out = self.dense_layer(out)
    return out


In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  tf.keras.utils.set_random_seed(42)
  layer = FullyNativeModel(10)

  inputs = tf.random.normal(shape=(10, 5, 5, 5))
  migrated_output = layer(inputs)

  # Grab the regularization loss as well
  migrated_regularization_loss = tf.math.add_n(layer.losses)

print(migrated_regularization_loss)

In [ ]:
# Verify that the regularization loss and output both match
np.testing.assert_allclose(original_regularization_loss.numpy(), migrated_regularization_loss.numpy())
np.testing.assert_allclose(original_output.numpy(), migrated_output.numpy())

### 네이티브 TF2로 마이그레이션하는 동안 체크포인트 호환성 유지하기

위의 네이티브 TF2 API로의 마이그레이션 프로세스는 변수 이름(Keras API가 매우 다른 가중치 이름을 생성하기 때문에)과 모델의 다른 가중치를 가리키는 객체 지향 경로를 모두 변경했습니다. 이러한 변경의 영향으로 기존 TF1 스타일 이름 기반 체크포인트 또는 TF2 스타일 객체 지향 체크포인트가 모두 망가질 수 있습니다.

그러나 경우에 따라 [TF1.x 체크포인트 가이드 재사용하기](./migrating_checkpoints.ipynb)에 자세히 설명된 것과 같은 접근 방식을 사용하여 원래 이름 기반 체크포인트를 사용하고 새 이름에 해당하는 변수 매핑을 찾을 수 있습니다.

이를 실현하는 몇 가지 팁은 다음과 같습니다.

- 여전히 변수에는 모두 설정할 수 있는 `name` 인수가 있습니다.
- 또한 Keras 모델은 변수의 접두사로 설정하는 `name` 인수를 사용합니다.
- v1.name_scope 함수를 변수 이름의 접두어를 지정하는데 사용할 수 있습니다. 이 함수는 tf.variable_scope와는 매우 다릅니다. 이름에만 영향을 미치며 변수를 추적하거나 재사용을 관장하지 않습니다.

위의 포인트터를 기반으로 다음 코드 샘플은 체크포인트를 업데이트하는 동시에 코드에 적용하여 모델의 일부를 점진적으로 업데이트할 수 있는 워크플로를 보여줍니다.

참고: Keras 레이어의 변수 이름 지정은 복잡하기 때문에 일부 사용 사례에서 작동하지 않을 수 있습니다.

1. 함수형 스타일 `tf.compat.v1.layers`를 객체 지향 버전으로 전환하는 작업으로 시작합니다.

In [ ]:
class FunctionalStyleCompatModel(tf.keras.layers.Layer):

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    with tf.compat.v1.variable_scope('model'):
      out = tf.compat.v1.layers.conv2d(
          inputs, 3, 3,
          kernel_regularizer="l2")
      out = tf.compat.v1.layers.conv2d(
          out, 4, 4,
          kernel_regularizer="l2")
      out = tf.compat.v1.layers.conv2d(
          out, 5, 5,
          kernel_regularizer="l2")
      return out

layer = FunctionalStyleCompatModel()
layer(tf.ones(shape=(10, 10, 10, 10)))
[v.name for v in layer.weights]

1. 그 다음에는 compat.v1.layer 객체와 `compat.v1.get_variable`로 생성한 모든 변수를 `track_tf1_style_variables`으로 메서드를 데코레이팅한 `tf.keras.layers.Layer`/`tf.Module` 객체의 속성으로 할당합니다(모든 객체 지향 TF2 스타일 체크포인트는 이제 변수 이름을 사용한 경로와 새로운 객체 지향 경로를 모두 저장).

In [ ]:
class OOStyleCompatModel(tf.keras.layers.Layer):

  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.conv_1 = tf.compat.v1.layers.Conv2D(
          3, 3,
          kernel_regularizer="l2")
    self.conv_2 = tf.compat.v1.layers.Conv2D(
          4, 4,
          kernel_regularizer="l2")

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    with tf.compat.v1.variable_scope('model'):
      out = self.conv_1(inputs)
      out = self.conv_2(out)
      out = tf.compat.v1.layers.conv2d(
          out, 5, 5,
          kernel_regularizer="l2")
      return out

layer = OOStyleCompatModel()
layer(tf.ones(shape=(10, 10, 10, 10)))
[v.name for v in layer.weights]

1. 이 시점에서 로드된 체크포인트를 다시 저장하여 변수 이름(compat.v1.layers의 경우) 또는 객체 지향 개체 그래프로 경로를 모두 저장합니다.

In [ ]:
weights = {v.name: v for v in layer.weights}
assert weights['model/conv2d/kernel:0'] is layer.conv_1.kernel
assert weights['model/conv2d_1/bias:0'] is layer.conv_2.bias

1. 이제 최근에 저장한 체크포인트를 계속 로드하는 동안에도 네이티브 Keras 레이어에서 객체 지향 `compat.v1.layers`를 교체할 수 있습니다. 교체된 레이어의 자동 생성된 `variable_scopes`를 계속 기록하여 남은 `compat.v1.layers`의 변수 이름을 보존해야 합니다. 이렇게 전환된 레이어/변수는 이제 변수 이름 경로 대신 체크포인트 변수의 객체 속성 경로만 사용합니다.

일반적으로 속성에 연결된 변수에서 `compat.v1.get_variable`의 사용을 다음과 같이 교체할 수 있습니다.

- `tf.Variable`을 사용하도록 전환, **또는**
- [`tf.keras.layers.Layer.add_weight`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer#add_weight)를 사용하여 업데이트. 한 번에 모든 레이어를 전환하지 않으면 `name` 인수가 누락된 남은 `compat.v1.layers`의 자동 생성된 레이어/변수 이름이 변경될 수 있습니다. 이 경우 제거된 `compat.v1.layer`의 생성된 범위 이름에 해당하는 `variable_scope`를 수동으로 열고 닫으며 나머지 `compat.v1.layers`의 변수 이름을 동일하게 유지해야 합니다. 그렇지 않으면 기존 체크포인트의 경로가 충돌하고 체크포인트 로드가 올바르지 않게 작동할 수 있습니다.


In [ ]:
def record_scope(scope_name):
  """Record a variable_scope to make sure future ones get incremented."""
  with tf.compat.v1.variable_scope(scope_name):
    pass

class PartiallyNativeKerasLayersModel(tf.keras.layers.Layer):

  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.conv_1 = tf.keras.layers.Conv2D(
          3, 3,
          kernel_regularizer="l2")
    self.conv_2 = tf.keras.layers.Conv2D(
          4, 4,
          kernel_regularizer="l2")

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    with tf.compat.v1.variable_scope('model'):
      out = self.conv_1(inputs)
      record_scope('conv2d') # Only needed if follow-on compat.v1.layers do not pass a `name` arg
      out = self.conv_2(out)
      record_scope('conv2d_1') # Only needed if follow-on compat.v1.layers do not pass a `name` arg
      out = tf.compat.v1.layers.conv2d(
          out, 5, 5,
          kernel_regularizer="l2")
      return out

layer = PartiallyNativeKerasLayersModel()
layer(tf.ones(shape=(10, 10, 10, 10)))
[v.name for v in layer.weights]

변수를 구성한 후 이 단계에서 체크포인트를 저장하면 현재 사용할 수 있는 ***객체 경로만*** 포함됩니다.

남은 `compat.v1.layers`의 자동 생성된 가중치 이름을 보존하려면 제거된 `compat.v1.layers`의 범위를 기록해야 합니다.

In [ ]:
weights = set(v.name for v in layer.weights)
assert 'model/conv2d_2/kernel:0' in weights
assert 'model/conv2d_2/bias:0' in weights

1. 모델의 모든 `compat.v1.layers` 및 `compat.v1.get_variable`을 완전한 네이티브 항목으로 교체할 때까지 위의 단계를 반복합니다.

In [ ]:
class FullyNativeKerasLayersModel(tf.keras.layers.Layer):

  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.conv_1 = tf.keras.layers.Conv2D(
          3, 3,
          kernel_regularizer="l2")
    self.conv_2 = tf.keras.layers.Conv2D(
          4, 4,
          kernel_regularizer="l2")
    self.conv_3 = tf.keras.layers.Conv2D(
          5, 5,
          kernel_regularizer="l2")


  def call(self, inputs, training=None):
    with tf.compat.v1.variable_scope('model'):
      out = self.conv_1(inputs)
      out = self.conv_2(out)
      out = self.conv_3(out)
      return out

layer = FullyNativeKerasLayersModel()
layer(tf.ones(shape=(10, 10, 10, 10)))
[v.name for v in layer.weights]

새로 업데이트한 체크포인트가 계속 예상대로 작동하는지 테스트해야 합니다. 마이그레이션한 코드가 올바르게 실행되도록 이 프로세스의 모든 증분 단계에서 [수치 정확성 검증 가이드](./validate_correctness.ipynb)에 설명된 기술을 적용합니다.

## 모델링 shim에서 다루지 않는 TF1.x에서 TF2 작업 변경 처리하기

이 가이드에 설명된 모델링 shim은 `get_variable`, `tf.compat.v1.layers`, `variable_scope` 의미 체계로 생성한 변수, 레이어 및 정규화 손실이 즉시 실행 및 `tf.function`을 사용할 때 컬렉션에 의존하지 않고 이전처럼 계속 작동하도록 할 수 있습니다.

여기에는 모델 순방향 전달이 의존할 수 있는 ***모든*** TF1.x에 특화된 의미 체계가 포함되지 않습니다. 경우에 따라 shim이 자체적으로 TF2에서 실행되는 모델 순방향 전달을 가져오기에 충분하지 않을 수 있습니다. TF1.x와 TF2의 동작 차이점에 대해 자세히 알아보려면 [TF1.x과 TF2 동작 차이 가이드](./tf1_vs_tf2)를 읽어보세요.